In [3]:
from dataset import Raw_PhysionNet,PSD_PhysioNet
from torch.utils.data import  random_split,DataLoader
import torch
from classifier import Deep_Classifier
import os
import config_local
import torch.nn as nn
import mne

In [41]:
def gen_dataloader(dataset,split_ratios):
    """
    Args:
        dataset: torch dataset
        split_ratio: list of floats that sums to one, represeting the size of train ,val and test_set 
    """

    train,val,test = random_split(dataset,lengths=split_ratios)

    train_loader = DataLoader(train,batch_size=32,shuffle=True)
    val_loader = DataLoader(val,batch_size=32,shuffle=False)
    test_loader = DataLoader(test,batch_size=32,shuffle=False)

    return train_loader,val_loader,test_loader

In [42]:
def train(model,train_loader,val_loader,optimizer,criterion,num_epochs, loss_train, loss_val, acc_val):

    for epoch in range(num_epochs):

        model.train()
        epoch_loss = 0
        for batch_data, batch_labels in train_loader:

            batch_data = batch_data.float()
            batch_labels = batch_labels.long()

            # print("batch_labels ",batch_labels.shape)

            if(batch_data.shape[0] == 1):
                continue # skipping any batchsize with only one example since batch normalization is being used
            # print("batch_data.shape 1",batch_data.shape)

            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            epoch_loss+= loss.item()
            loss.backward()
            optimizer.step()

        # saving the epoch loss
        epoch_loss = epoch_loss/len(train_loader)
        loss_train.append(epoch_loss)

        # Validation loop
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            correct = 0
            total = 0
            for batch_data, batch_labels in val_loader:
                batch_data = batch_data.float()
                batch_labels = batch_labels.long()
                
                outputs = model(batch_data)
                _, predicted = torch.max(outputs, 1)
                total += batch_labels.size(0)
                correct += (predicted == batch_labels).sum().item()
                val_loss += criterion(outputs, batch_labels).item()
            
            val_loss /= len(val_loader)
            accuracy = correct / total
            
            loss_val.append(val_loss)
            acc_val.append(accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {accuracy:.2f}')
    
    return loss_train,loss_val,acc_val


In [43]:
def test(model,test_loader,criterion):

    model.eval()

    prdicted_labels = [] # list of all predicted labels

    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for batch_data, batch_labels in test_loader:
            outputs = model(batch_data.float())
            _, predicted = torch.max(outputs, 1)
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
            val_loss += criterion(outputs, batch_labels.long()).item()

            prdicted_labels = prdicted_labels + list(predicted) 

        val_loss /= len(test_loader)
        accuracy = correct / total
    
    
    print(f'Test Loss: {val_loss:.4f}, Test Acc: {accuracy:.2f}')

    return val_loss , accuracy , prdicted_labels


In [1]:
def cosine_similarity_matrix(vectors):
    """
    Computes the cosine similarity matrix for a set of vectors.

    Parameters:
    vectors (torch.Tensor): A torch tensor of shape (n, 128) representing n vectors.

    Returns:
    torch.Tensor: An n x n matrix where element (i, j) represents the cosine similarity between vector i and vector j.
    """
    # Normalize the vectors
    norm_vectors = vectors / vectors.norm(dim=1, keepdim=True)
    
    # Compute the cosine similarity matrix
    similarity_matrix = torch.mm(norm_vectors, norm_vectors.T)
    
    return similarity_matrix

In [4]:
vecs = torch.rand((50,128))
mat = cosine_similarity_matrix(vecs)

In [15]:
mat[2,3]

tensor(0.7503)

In [19]:
import itertools

In [22]:


def generate_combinations(elements, length):
    """
    Generates all possible combinations of a given length from a list of elements.

    Parameters:
    elements (list): The list of elements to combine.
    length (int): The length of each combination.

    Returns:
    list: A list of lists, where each inner list is a combination.
    """
    return np.array(list(itertools.product(elements, repeat=length)))

# Example usage
elements = [1, 2]
length = 2

combinations = generate_combinations(elements, length)
print(combinations)

[[1 1]
 [1 2]
 [2 1]
 [2 2]]


In [27]:
mat[tuple(combinations[0])]

tensor(1.0000)

In [17]:
np.array([1,2,3])-np.array([2,3])

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [14]:
import numpy as np
np.mean([1,2,3])

2.0

In [8]:
x = torch.randint(low=1,high=10,size=(128,1))

In [10]:
import torch
from collections import defaultdict

def group_indices(tensor):
    groups = defaultdict(list)
    for idx, item in enumerate(tensor):
        groups[item.item()].append(idx)
    return list(groups.values())

# Example usage
input_tensor = torch.tensor([2, 2, 3, 1, 2, 3])
result = group_indices(input_tensor)
print(result)  # Output: [[0, 3], [1, 4], [2, 5]]

[[0, 1, 4], [2, 5], [3]]


In [44]:
NUM_SUBJS = int(os.getenv("NUM_SUBJS"))
bin_dict_extnd = {
    "delta":[1,4],
    "theta":[4,8],
    "alpha":[8,13],
    "beta":[13,30],
    "gamma":[30,79]
}


In [45]:
model = Deep_Classifier(num_classes=NUM_SUBJS)
criterion = nn.CrossEntropyLoss()

In [46]:
raw_dataset = Raw_PhysionNet()
# psd_dataset = PSD_PhysioNet(raw_dataset=raw_dataset,freq_bin=bin_dict_extnd,duration=1)

.... found 1526 edf files ....


In [47]:
raw_dataset.eeg_data_x.shape

(4706, 64, 321)

In [48]:
raw_dataset.eeg_data_x.shape

(4706, 64, 321)

In [49]:
raw_dataset.__len__()

18824

In [50]:
train_loader,val_loader,test_loader =gen_dataloader(raw_dataset,[0.7,0.15,0.15])

In [51]:
for i,j in train_loader:
    print(j)
    break

tensor([ 4,  3,  2, 12,  3,  2,  9,  2,  4,  6,  9, 12,  8, 11,  0, 11, 11,  1,
         0,  0,  2,  2, 11, 10, 11,  7,  2,  6,  7, 12,  3,  8])


In [52]:
64*5

320

In [53]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [54]:
loss_train=[]
loss_val=[]
acc_val=[]

In [55]:
for i in train_loader:
    print(i[0].shape)
    break

torch.Size([32, 64, 80])


In [56]:
model

Deep_Classifier(
  (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (point_wise): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6144, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=13, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (relu): ReLU()
)

In [57]:
train(model,train_loader,val_loader,optimizer,criterion,1000,loss_train,loss_val,acc_val)

out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])
out.shape  torch.Size([32, 6144])


KeyboardInterrupt: 